最初に使いそうなものをimportしておきます。

In [ ]:
import math

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot

In [ ]:
from PIL import Image

In [ ]:
from google.colab import drive

自分のドライブにアクセスできるようにします。Googleアカウントが必要です。

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


画像ファイルを自分のドライブにコピーします。

In [ ]:
!wget -O drive/MyDrive/number.jpg https://github.com/toho-cmt/brain/raw/main/number.jpg
!ls -l drive/MyDrive/number.jpg

画像ファイルを読み込みます。

In [ ]:
file_orig = 'drive/MyDrive/number.jpg'

In [ ]:
image = Image.open(file_orig) #ドライブから画像を読み込みます。

読み込んだファイルを表示してみます。

In [ ]:
image_large = image.resize((image.width*5, image.height*5),resample=0)
image_large

In [ ]:
from PIL import ImageOps

画像がカラーの場合はグレースケールに変換してから行列に変換します。

In [ ]:
image_gray = ImageOps.grayscale(image)

In [ ]:
A = np.array(image_gray, dtype=float)

行列のサイズを確認します。

In [ ]:
print('size of A:', A.shape)

size of A: (28, 200)


In [ ]:
A = np.uint8(A) #行列の値を整数にします。

In [ ]:
A = (A//128)*255 #白（255）か黒(0)の2値に変換します。

白と黒のみで表した画像を確認してみます。

In [ ]:
image_binary = Image.fromarray(A)

In [ ]:
image_binary_large = image_binary.resize((image_binary.width*5, image_binary.height*5),resample=0)
image_binary_large

In [ ]:
A = (A//255 -0.5)*2 #行列の成分を255（白）と0（黒）から1と-1に変換しておきます。

初期画像として、各ピクセルに白と黒をランダムに配置したモザイク模様を採用します。

In [ ]:
Ainit = np.random.randint(255, size=(28,200),dtype=np.uint8)

乱数は０〜２５５の値を一様にとる乱数なので、それを128以上を255(白)、それ以下の値を黒（0）にします。

In [ ]:
Ainit = (Ainit//128)*255

初期画像を表示してみると以下のようなものになります。

In [ ]:
imageinit = Image.fromarray(Ainit)

In [ ]:
imageinit_large = imageinit.resize((imageinit.width*5, imageinit.height*5),resample=0)
imageinit_large

In [ ]:
Ainit = (Ainit//255 -0.5)*2 #255（白）と0（黒）を1と-1に変換します。

In [ ]:
A_rev = np.zeros((28,200)) #更新する画像の初期化を行います。

In [ ]:
for i in range(28):
 for j in range(200):
  for k in range (28):
   for l in range (200):
    A_rev[i,j] += A[i,j]*A[k,l]*Ainit[k,l] #更新開始、まず、Jijをかけて和をとる

In [ ]:
for i in range(28):
 for j in range (200):
  A_rev[i,j] -= A[i,j]*A[i,j]*Ainit[i,j] #Jiiの対角成分を引く

In [ ]:
A_rev = np.sign(A_rev) #和をとったものの符号を求める。これで更新１回目終了。

In [ ]:
A_rev = (A_rev+1.1)//2*255 #更新した画像で、1(-1)を255白(0黒)に変換。

ランダムなモザイク模様の初期画像が、１回の更新で以下のようになります。

In [ ]:
image_rev = Image.fromarray(np.uint8(A_rev))

In [ ]:
image_rev_large = image_rev.resize((image_rev.width*5, image_rev.height*5),resample=0)
image_rev_large

In [ ]:
file_out = 'drive/MyDrive/image_out.jpg'

In [ ]:
imageinit_large.save(file_out)